Based off of: https://www.kaggle.com/code/pavansanagapati/ensemble-learning-techniques-tutorial

In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

import pandas as pd

In [2]:

housing = pd.read_csv('/kaggle/input/housing/houseprice.csv'',na_values = '#NAME?')


SyntaxError: incomplete input (<ipython-input-2-00e34c274bb7>, line 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.20,random_state = 42) 